In [27]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from cmapPy.pandasGEXpress.parse import parse

from scipy.stats import spearmanr as scor
from scipy.stats import pearsonr
from scipy.stats import mannwhitneyu as mwu

from sklearn.metrics import roc_curve as ROC
from sklearn.metrics import auc as AUC
from sklearn.metrics import precision_recall_curve as PRC

# Importing data for comparison

In [37]:
#Importing CREEDs benchmark data files 
gene_perturb=pd.read_csv('../results/gene_pert.csv', sep=',', index_col=0, header=0)
drug_perturb=pd.read_csv('../results/drug_pert.csv', sep=',', index_col=0, header=0)

In [38]:
#Checking the absolut values of the benchmark data also
gene_perturb_A=gene_perturb.abs()
drug_perturb_A=drug_perturb.abs()
gene_perturb_A.to_csv('../results/gene_pert_A.csv', sep=',')
drug_perturb_A.to_csv('../results/drug_pert_A.csv', sep=',')

In [50]:
#Importing the NicheNet model
nn_model=pd.read_csv('../data/ligand_target_matrix.csv', sep=',', header=0, index_col=0)
nn_model = nn_model.reindex(sorted(nn_model.columns), axis=1)
ligand_receptor=pd.read_csv('../data/lr_network.csv', sep=',', header=0, index_col=0)

In [40]:
#Importing LINCS perturbation signatures 
consig_gse70138_trt_cp=pd.read_csv('../results/consensus_signature_gse70138_trt_cp.csv', header=0, sep=',', index_col=0)
consig_gse70138_trt_xpr=pd.read_csv('../results/consensus_signature_gse70138_trt_xpr.csv', header=0, sep=',', index_col=0)
consig_gse92742_trt_cp=pd.read_csv('../results/consensus_signature_gse92742_trt_cp.csv', header=0, sep=',', index_col=0)
consig_gse92742_trt_lig=pd.read_csv('../results/consensus_signature_gse92742_trt_lig.csv', header=0, sep=',', index_col=0)
consig_gse92742_trt_oe=pd.read_csv('../results/consensus_signature_gse92742_trt_oe.csv', header=0, sep=',', index_col=0)
consig_gse92742_trt_sh=pd.read_csv('../results/consensus_signature_gse92742_trt_sh.csv', header=0, sep=',', index_col=0)
consig_gse92742_trt_sh_fil=pd.read_csv('../results/consensus_signature_gse92742_trt_sh_fil.csv', header=0, sep=',', index_col=0)

In [41]:
#Checking the absolut values of LINCS also
consig_gse70138_trt_cp_A=consig_gse70138_trt_cp.abs()
consig_gse70138_trt_xpr_A=consig_gse70138_trt_xpr.abs()
consig_gse92742_trt_cp_A=consig_gse92742_trt_cp.abs()
consig_gse92742_trt_lig_A=consig_gse92742_trt_lig.abs()
consig_gse92742_trt_oe_A=consig_gse92742_trt_oe.abs()
consig_gse92742_trt_sh_A=consig_gse92742_trt_sh.abs()
consig_gse92742_trt_sh_fil_A=consig_gse92742_trt_sh_fil.abs()

In [42]:
consig_gse70138_trt_xpr=consig_gse70138_trt_xpr.T
consig_gse92742_trt_lig=consig_gse92742_trt_lig.T
consig_gse92742_trt_oe=consig_gse92742_trt_oe.T
consig_gse92742_trt_sh=consig_gse92742_trt_sh.T
consig_gse92742_trt_sh_fil=consig_gse92742_trt_sh_fil.T

consig_gse70138_trt_xpr_A=consig_gse70138_trt_xpr_A.T
consig_gse92742_trt_lig_A=consig_gse92742_trt_lig_A.T
consig_gse92742_trt_oe_A=consig_gse92742_trt_oe_A.T
consig_gse92742_trt_sh_A=consig_gse92742_trt_sh_A.T
consig_gse92742_trt_sh_fil_A=consig_gse92742_trt_sh_fil_A.T

# NicheNet vs. CREEDs

### NN vs. drug_perturb

In [49]:
drug_perturb.shape

(7781, 387)

In [50]:
nn_model.shape

(25345, 688)

In [51]:
common_genes=list(set(nn_model.index)& set(drug_perturb.index))

In [52]:
len(common_genes)

7727

In [53]:
nn_model=nn_model.loc[common_genes]
drug_perturb=drug_perturb.loc[common_genes]
drug_perturb_A=drug_perturb_A.loc[common_genes]

In [62]:
drug_perturb.shape

(7727, 387)

In [55]:
nn_model.shape

(7727, 688)

In [56]:
drug_perturb.head()

,1416,1074,925,345,551,750,885,1584,1585,636,...,1912,552,1362,668,697,1369,550,1579,1219,1514
FAM3A,1.528467,-0.150743,0.222181,-0.470496,0.731880,-0.181326,3.436021,-1.500258,-1.500258,0.068688,...,-0.054181,-0.471603,1.638888,0.813216,1.220660,7.955420,2.458027,-1.388246,0.192524,0.089808
MPI,-0.349662,-2.366964,-0.065323,0.101187,-0.379393,0.432761,3.318031,1.594171,1.594171,-0.759975,...,1.402696,-0.200662,-1.609156,2.248813,0.453140,2.587244,2.098900,1.275870,-0.174179,-0.738923
MIER2,-1.125718,0.779621,-0.603046,-0.580376,0.836180,1.459272,1.060877,0.616406,0.616406,-0.509168,...,1.237060,-3.720114,-0.269386,-0.343174,0.976845,-0.602606,-1.084516,1.839062,-1.892136,0.419547
ARHGEF10,-0.178789,0.251414,-1.485443,0.183717,0.564224,1.062560,-1.319166,-0.834716,-0.834716,-0.852956,...,-1.157459,-1.534592,-1.136904,-1.125984,-0.257826,8.268007,1.298609,0.539243,-1.166287,0.863441
SPATA24,1.005510,-2.834021,3.133430,-1.404423,0.773887,-0.140430,-0.627203,1.616636,1.616636,-0.366637,...,2.014616,1.198471,-0.936847,-0.579943,-4.438929,-10.927265,-0.068208,-0.686556,-0.744657,0.740323


In [67]:
drug_NN=pd.DataFrame(index=nn_model.columns, columns=drug_perturb.columns)
drug_A_NN=pd.DataFrame(index=nn_model.columns, columns=drug_perturb_A.columns)

In [68]:
drug_NN.head()

,1416,1074,925,345,551,750,885,1584,1585,636,...,1912,552,1362,668,697,1369,550,1579,1219,1514
A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AANAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADAM12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADAM15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADAM17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
for lig in drug_NN.index:
    for exp in drug_NN.columns:
        r=pearsonr(nn_model[lig],drug_perturb[exp])
        r=r[0]
        drug_NN.loc[lig,exp]=r

C:\Users\nidik\anaconda3\lib\site-packages\scipy\stats\stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [70]:
for lig in drug_A_NN.index:
    for exp in drug_A_NN.columns:
        r=pearsonr(nn_model[lig],drug_perturb_A[exp])
        r=r[0]
        drug_A_NN.loc[lig,exp]=r

In [72]:
drug_NN.to_csv('../results/drug_NN_sim.csv', sep=',')
drug_A_NN.to_csv('../results/drug_A_NN_sim.csv', sep=',')

### NN vs. gene_perturb

In [73]:
common_genes=list(set(nn_model.index)& set(gene_perturb.index))
len(common_genes)

6826

In [74]:
nn_model=nn_model.loc[common_genes]
gene_perturb=gene_perturb.loc[common_genes]
gene_perturb_A=gene_perturb_A.loc[common_genes]

In [75]:
gene_NN=pd.DataFrame(index=nn_model.columns, columns=gene_perturb.columns)
gene_A_NN=pd.DataFrame(index=nn_model.columns, columns=gene_perturb_A.columns)

In [76]:
gene_NN.head()

,2410,2860,2487,2723,1319,3062,169,358,2422,2338,...,1830,2025,2777,2993,1946,2389,378,2411,2982,2742
A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AANAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADAM12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADAM15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADAM17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
for lig in gene_NN.index:
    for exp in gene_NN.columns:
        r=pearsonr(nn_model[lig],gene_perturb[exp])
        r=r[0]
        gene_NN.loc[lig,exp]=r

C:\Users\nidik\anaconda3\lib\site-packages\scipy\stats\stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [78]:
for lig in gene_A_NN.index:
    for exp in gene_A_NN.columns:
        r=pearsonr(nn_model[lig],gene_perturb_A[exp])
        r=r[0]
        gene_A_NN.loc[lig,exp]=r

In [80]:
gene_NN.to_csv('../results/gene_NN_sim.csv', sep=',')
gene_A_NN.to_csv('../results/gene_A_NN_sim.csv', sep=',')

# LINCS vs. CREEDs

In [43]:
def lincs_creeds_sim(consig, creeds_name, fname):
    creeds=pd.read_csv('../results/'+creeds_name+'.csv',sep=',', index_col=0, header=0)
    common_genes=list(set(consig.index)& set(creeds.index))
    creeds=creeds.loc[common_genes]
    consig=consig.loc[common_genes]
    similarity_fname=pd.DataFrame(index=consig.columns, columns=creeds.columns)
    for lig in similarity_fname.index:
        for exp in similarity_fname.columns:
            r=pearsonr(consig[lig],creeds[exp])
            r=r[0]
            similarity_fname.loc[lig,exp]=r
    similarity_fname.to_csv('../results/'+creeds_name+'_'+fname+'.csv', sep=',')
    #return similarity_fname.head()

### LINCS vs. drug_perturb

In [45]:
lincs_creeds_sim(consig_gse70138_trt_xpr, 'drug_pert', 'gse70138_trt_xpr')
lincs_creeds_sim(consig_gse70138_trt_xpr_A, 'drug_pert', 'gse70138_trt_xpr_A')
lincs_creeds_sim(consig_gse92742_trt_lig, 'drug_pert', 'gse70138_trt_lig')
lincs_creeds_sim(consig_gse92742_trt_lig_A, 'drug_pert', 'gse70138_trt_lig_A')
lincs_creeds_sim(consig_gse92742_trt_oe, 'drug_pert', 'gse70138_trt_oe')
lincs_creeds_sim(consig_gse92742_trt_oe_A, 'drug_pert', 'gse70138_trt_oe_A')
lincs_creeds_sim(consig_gse92742_trt_sh, 'drug_pert', 'gse70138_trt_sh')
lincs_creeds_sim(consig_gse92742_trt_sh_A, 'drug_pert', 'gse70138_trt_sh_A')
lincs_creeds_sim(consig_gse92742_trt_sh_fil, 'drug_pert', 'gse70138_trt_sh_fil')
lincs_creeds_sim(consig_gse92742_trt_sh_fil_A, 'drug_pert', 'gse70138_trt_sh_fil_A')

### LINCS vs. drug_perturb_A

In [46]:
lincs_creeds_sim(consig_gse70138_trt_xpr, 'drug_pert_A', 'gse70138_trt_xpr')
lincs_creeds_sim(consig_gse70138_trt_xpr_A, 'drug_pert_A', 'gse70138_trt_xpr_A')
lincs_creeds_sim(consig_gse92742_trt_lig, 'drug_pert_A', 'gse70138_trt_lig')
lincs_creeds_sim(consig_gse92742_trt_lig_A, 'drug_pert_A', 'gse70138_trt_lig_A')
lincs_creeds_sim(consig_gse92742_trt_oe, 'drug_pert_A', 'gse70138_trt_oe')
lincs_creeds_sim(consig_gse92742_trt_oe_A, 'drug_pert_A', 'gse70138_trt_oe_A')
lincs_creeds_sim(consig_gse92742_trt_sh, 'drug_pert_A', 'gse70138_trt_sh')
lincs_creeds_sim(consig_gse92742_trt_sh_A, 'drug_pert_A', 'gse70138_trt_sh_A')
lincs_creeds_sim(consig_gse92742_trt_sh_fil, 'drug_pert_A', 'gse70138_trt_sh_fil')
lincs_creeds_sim(consig_gse92742_trt_sh_fil_A, 'drug_pert_A', 'gse70138_trt_sh_fil_A')

### LINCS vs. gene_perturb

In [47]:
lincs_creeds_sim(consig_gse70138_trt_xpr, 'gene_pert', 'gse70138_trt_xpr')
lincs_creeds_sim(consig_gse70138_trt_xpr_A, 'gene_pert', 'gse70138_trt_xpr_A')
lincs_creeds_sim(consig_gse92742_trt_lig, 'gene_pert', 'gse70138_trt_lig')
lincs_creeds_sim(consig_gse92742_trt_lig_A, 'gene_pert', 'gse70138_trt_lig_A')
lincs_creeds_sim(consig_gse92742_trt_oe, 'gene_pert', 'gse70138_trt_oe')
lincs_creeds_sim(consig_gse92742_trt_oe_A, 'gene_pert', 'gse70138_trt_oe_A')
lincs_creeds_sim(consig_gse92742_trt_sh, 'gene_pert', 'gse70138_trt_sh')
lincs_creeds_sim(consig_gse92742_trt_sh_A, 'gene_pert', 'gse70138_trt_sh_A')
lincs_creeds_sim(consig_gse92742_trt_sh_fil, 'gene_pert', 'gse70138_trt_sh_fil')
lincs_creeds_sim(consig_gse92742_trt_sh_fil_A, 'gene_pert', 'gse70138_trt_sh_fil_A')

### LINCS vs. gene_perturb_A

In [48]:
lincs_creeds_sim(consig_gse70138_trt_xpr, 'gene_pert_A', 'gse70138_trt_xpr')
lincs_creeds_sim(consig_gse70138_trt_xpr_A, 'gene_pert_A', 'gse70138_trt_xpr_A')
lincs_creeds_sim(consig_gse92742_trt_lig, 'gene_pert_A', 'gse70138_trt_lig')
lincs_creeds_sim(consig_gse92742_trt_lig_A, 'gene_pert_A', 'gse70138_trt_lig_A')
lincs_creeds_sim(consig_gse92742_trt_oe, 'gene_pert_A', 'gse70138_trt_oe')
lincs_creeds_sim(consig_gse92742_trt_oe_A, 'gene_pert_A', 'gse70138_trt_oe_A')
lincs_creeds_sim(consig_gse92742_trt_sh, 'gene_pert_A', 'gse70138_trt_sh')
lincs_creeds_sim(consig_gse92742_trt_sh_A, 'gene_pert_A', 'gse70138_trt_sh_A')
lincs_creeds_sim(consig_gse92742_trt_sh_fil, 'gene_pert_A', 'gse70138_trt_sh_fil')
lincs_creeds_sim(consig_gse92742_trt_sh_fil_A, 'gene_pert_A', 'gse70138_trt_sh_fil_A')

# Evaluating similarity with ROC curves

### Comparing NicheNet with CREEDs

In [89]:
similarity=pd.read_csv('../results/drug_NN_sim.csv',sep=',',header=0,index_col=0)

In [91]:
similarity

,1416,1074,925,345,551,750,885,1584,1585,636,...,1912,552,1362,668,697,1369,550,1579,1219,1514
A2M,0.014391,0.019366,-0.037563,-0.029965,0.077882,0.036462,0.050678,0.024175,0.024175,-0.030569,...,-0.149099,-0.065063,0.052309,0.098862,0.052746,-0.008835,0.047937,0.012386,-0.039576,0.107670
AANAT,-0.000206,0.023141,-0.029914,-0.034754,0.058187,0.061019,0.089336,0.055806,0.055806,-0.044222,...,-0.114710,-0.023075,0.034852,0.068535,0.032449,0.020715,0.071374,0.029612,-0.040648,0.118762
ADAM12,0.018130,0.028461,-0.043781,-0.035015,0.079224,0.026426,0.027200,0.006472,0.006472,-0.014927,...,-0.146156,-0.077751,0.067420,0.097866,0.043083,-0.024648,0.038684,0.010635,-0.040301,0.089911
ADAM15,0.003441,0.026603,-0.000578,-0.045290,0.064008,0.070735,0.069970,0.050979,0.050979,-0.043821,...,-0.112256,-0.054955,0.048894,0.085942,0.014557,0.018594,0.059385,0.020237,-0.035226,0.102608
ADAM17,0.010067,0.000684,-0.035540,-0.039221,0.051589,0.036043,0.064493,0.030576,0.030576,-0.025098,...,-0.140735,-0.063715,0.047572,0.077192,0.063302,0.006946,0.036739,0.018004,-0.036609,0.107524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WNT9B,0.011905,0.028673,-0.026329,-0.034678,0.076628,0.049257,0.036342,0.019108,0.019108,-0.027842,...,-0.144827,-0.074014,0.059421,0.100202,0.039309,-0.001753,0.040816,0.011912,-0.043135,0.098178
XCL1,0.010616,0.024755,-0.042218,-0.021946,0.077166,0.043705,0.048979,0.024001,0.024001,-0.025184,...,-0.154350,-0.063929,0.053586,0.095332,0.067670,0.004028,0.036146,0.009512,-0.045133,0.104812
XCL2,0.012008,0.025227,-0.035040,-0.021158,0.077544,0.043534,0.042630,0.020494,0.020494,-0.022756,...,-0.152779,-0.064043,0.048061,0.101015,0.062870,0.001851,0.034304,0.010355,-0.045834,0.101652
YARS,0.011136,0.023499,-0.017143,-0.034719,0.058188,0.047371,0.014746,0.003287,0.003287,-0.010136,...,-0.142318,-0.085313,0.053580,0.087443,0.045126,-0.002829,0.011195,0.006701,-0.040057,0.081317


In [90]:
for l_or_r in similarity.columns:
        pairs=[l_or_r]
        pairs+=list(ligand_receptor[ligand_receptor['to']==l_or_r]['from'])
        pairs=list(set(pairs)&set(truedata.index))
        truedata.loc[pairs,l_or_r]=1

In [ ]:
truedata=np.zeros((correlations.shape),int) #array
truedata=pd.DataFrame(truedata, index=correlations.index, columns=correlations.columns)

In [ ]:
def NN_compare(similarity_name) :
    similarity=pd.read_csv('../results/'+similarity+'.csv',sep=',',header=0,index_col=0)
    truedata=np.zeros((similarity.shape),int) #array
    truedata=pd.DataFrame(truedata, index=similarity.index, columns=similarity.columns)
    for l_or_r in similarity.columns:
        pairs=[l_or_r]
        pairs+=list(ligand_receptor[ligand_receptor['to']==l_or_r]['from'])
        pairs=list(set(pairs)&set(truedata.index))
        truedata.loc[pairs,l_or_r]=1
    
    truedata_ROC=truedata.values.reshape((1,-1))[0]
    #ROC
    y_pr=similarity
    y_true=truedata
    fpr, tpr, th=ROC(y_true=y_true, y_score=y_pr)
    plt.subplot(121)
    plt.plot(fpr, tpr)
    plt.xlabel('False positive rate', size=15)
    plt.ylabel('True positive rate', size=15)
    plt.title(similarity_name, size=12)
    plt.legend(['ROC AUC=' + str(AUC(fpr, tpr))[:4]])
    plt.tight_layout()
    print(AUC(fpr, tpr))
    #PRC
    y_pred=similarity
    y_tr=truedata
    precision, recall, thresholds = PRC( y_true=y_tr, probas_pred=y_pred)
    plt.subplot(122)
    plt.plot(recall, precision)
    plt.xlabel('Recall', size=15)
    plt.ylabel('Precision', size=15)
    plt.title(similarity_name, size=12)
    random_AUC=np.sum(y_tr)/len(y_tr)
    real_AUC=AUC(recall, precision)
    plt.plot([0,1],[random_AUC,random_AUC], 'k:') #matplotlib colours, line stile
    print (AUC(recall,precision))
    print('Random AUC:',np.sum(y_tr)/len(y_tr))
    plt.legend(['PRC AUC=' + str(real_AUC)[:4], 'random AUC='+ str(random_AUC)[:4]])
    plt.tight_layout()

In [ ]:
NN_compare('drug_NN_sim')

In [ ]:
NN_compare('drug_A_NN_sim')

In [ ]:
NN_compare('gene_NN_sim')

In [ ]:
NN_compare('gene_A_NN_sim')

### Comparing LINCS with CREEDs

In [ ]:
def LINCS_compare(similarity_name) :
    similarity=pd.read_csv('../results/'+similarity+'.csv',sep=',',header=0,index_col=0)
    
    #ROC
    y_pr=similarity
    y_true=pd.Series(0,index=y_pr.index)
    y_true[list(set(rl[pathway])&(set(progeny_consig.index)))]=1
    
    fpr, tpr, th=ROC(y_true=y_true, y_score=y_pr)
    plt.subplot(121)
    plt.plot(fpr, tpr)
    plt.xlabel('False positive rate', size=15)
    plt.ylabel('True positive rate', size=15)
    plt.title(similarity_name, size=12)
    plt.legend(['ROC AUC=' + str(AUC(fpr, tpr))[:4]])
    plt.tight_layout()
    print(AUC(fpr, tpr))
    #PRC
    y_pred=similarity
    y_tr=truedata
    precision, recall, thresholds = PRC( y_true=y_tr, probas_pred=y_pred)
    plt.subplot(122)
    plt.plot(recall, precision)
    plt.xlabel('Recall', size=15)
    plt.ylabel('Precision', size=15)
    plt.title(similarity_name, size=12)
    random_AUC=np.sum(y_tr)/len(y_tr)
    real_AUC=AUC(recall, precision)
    plt.plot([0,1],[random_AUC,random_AUC], 'k:') #matplotlib colours, line stile
    print (AUC(recall,precision))
    print('Random AUC:',np.sum(y_tr)/len(y_tr))
    plt.legend(['PRC AUC=' + str(real_AUC)[:4], 'random AUC='+ str(random_AUC)[:4]])
    plt.tight_layout()

In [ ]:
lincs_compare('drug_pert_gse70138_trt_xpr')
lincs_compare('drug_pert_gse70138_trt_xpr_A')
lincs_compare('drug_pert_gse70138_trt_lig')
lincs_compare('drug_pert_gse70138_trt_lig_A')
lincs_compare('drug_pert_gse70138_trt_oe')
lincs_compare('drug_pert_gse70138_trt_oe_A')
lincs_compare('drug_pert_gse70138_trt_sh')
lincs_compare('drug_pert_gse70138_trt_sh_A')
lincs_compare('drug_pert_gse70138_trt_sh_fil')
lincs_compare('drug_pert_gse70138_trt_sh_fil_A')

In [ ]:
lincs_compare('drug_pert_A_gse70138_trt_xpr')
lincs_compare('drug_pert_A_gse70138_trt_xpr_A')
lincs_compare('drug_pert_A_gse70138_trt_lig')
lincs_compare('drug_pert_A_gse70138_trt_lig_A')
lincs_compare('drug_pert_A_gse70138_trt_oe')
lincs_compare('drug_pert_A_gse70138_trt_oe_A')
lincs_compare('drug_pert_A_gse70138_trt_sh')
lincs_compare('drug_pert_A_gse70138_trt_sh_A')
lincs_compare('drug_pert_A_gse70138_trt_sh_fil')
lincs_compare('drug_pert_A_gse70138_trt_sh_fil_A')

In [ ]:
lincs_compare('gene_pert_gse70138_trt_xpr')
lincs_compare('gene_pert_gse70138_trt_xpr_A')
lincs_compare('gene_pert_gse70138_trt_lig')
lincs_compare('gene_pert_gse70138_trt_lig_A')
lincs_compare('gene_pert_gse70138_trt_oe')
lincs_compare('gene_pert_gse70138_trt_oe_A')
lincs_compare('gene_pert_gse70138_trt_sh')
lincs_compare('gene_pert_gse70138_trt_sh_A')
lincs_compare('gene_pert_gse70138_trt_sh_fil')
lincs_compare('gene_pert_gse70138_trt_sh_fil_A')

In [ ]:
lincs_compare('gene_pert_A_gse70138_trt_xpr')
lincs_compare('gene_pert_A_gse70138_trt_xpr_A')
lincs_compare('gene_pert_A_gse70138_trt_lig')
lincs_compare('gene_pert_A_gse70138_trt_lig_A')
lincs_compare('gene_pert_A_gse70138_trt_oe')
lincs_compare('gene_pert_A_gse70138_trt_oe_A')
lincs_compare('gene_pert_A_gse70138_trt_sh')
lincs_compare('gene_pert_A_gse70138_trt_sh_A')
lincs_compare('gene_pert_A_gse70138_trt_sh_fil')
lincs_compare('gene_pert_A_gse70138_trt_sh_fil_A')